CHARGEMENT DES LIBRAIRIES ET LE DATASET

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Pour visualiser rapidement les infos du dataset
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Versions (reproductibilité)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)

# Charger le dataset
df = pd.read_excel("../data/raw/OnlineRetail.xlsx")

# Afficher les premières lignes
df.head(6)

pandas version: 2.3.3
numpy version: 2.3.5


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom


COMPRÉHENSION DU DATASET

COMPRENDRE LES DONNÉES

In [3]:
# Informations générales
df.info()

# Statistiques descriptives
df.describe()

# Vérification des doublons
print("Nombre de doublons initiaux :", df.duplicated().sum())

# Valeurs manquantes
df.isnull().sum()

# Colonnes initiales
print("Colonnes brutes :", df.columns.tolist())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB
Nombre de doublons initiaux : 6865
Colonnes brutes : ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


STANDARDISATION DES NOMS DE COLONNES

NETTOYER LES DONNÉES

In [4]:
# Standardisation des noms de colonnes pour cohérence du projet
# Standardisation des noms de colonnes pour le projet
df.rename(columns={
    'Invoice': 'InvoiceNo',
    'Price': 'UnitPrice',
    'Customer ID': 'Customer_ID'
}, inplace=True)

print("Colonnes après renommage :", df.columns.tolist())


Colonnes après renommage : ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Customer_ID', 'Country']


NETTOYAGE DES DONNÉES

In [5]:
# Suppression des factures annulées (InvoiceNo commençant par 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [6]:
# Suppression des lignes sans Customer_ID
df = df.dropna(subset=['Customer_ID'])

# Conversion du type
df['Customer_ID'] = df['Customer_ID'].astype(int)

In [7]:
# Correction du type de Date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

GESTION DES DOUBLONS

In [8]:
# Suppression des doublons exacts
df = df.drop_duplicates()

print("Nombre de lignes après suppression des doublons :", len(df))

Nombre de lignes après suppression des doublons : 400947


CRÉATION DE LA MÉTRIQUE BUSINESS CLÉ: REVENUE

VÉRIFIER LES ANOMALIES

In [9]:
# Calcul du chiffre d'affaires par ligne
df['Revenue'] = df['Quantity'] * df['UnitPrice']

TRAITEMENT DES VALEURS INCOHÉRENTS

In [10]:
# Détection des revenus négatifs ou nuls
neg_revenue = df[df['Revenue'] <= 0]
print("Nombre de ventes négatives ou nulles :", len(neg_revenue))

# Suppression pour analyses business
df = df[df['Revenue'] > 0]

Nombre de ventes négatives ou nulles : 31


EXPORT DES DONNÉES NETTOYÉES

EXPORTER LES DONNÉES NETTOYÉES

In [11]:
# Chemin racine du projet
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Création du dossier processed si nécessaire
processed_path = os.path.join(project_root, "data", "processed")
os.makedirs(processed_path, exist_ok=True)

# Sauvegarde
csv_file_path = os.path.join(processed_path, "online_retail_clean.csv")
df.to_csv(csv_file_path, index=False)

print(f"Fichier nettoyé sauvegardé dans : {csv_file_path}")


Fichier nettoyé sauvegardé dans : c:\dev\Retail_Sales_Analysis_Project\data\processed\online_retail_clean.csv
